In [41]:
import pandas as pd
import datetime

In [42]:
# today's data and yesterday's date
today = (datetime.datetime.today().strftime('%d%b%Y')).upper()
yesterday = ((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%d%b%Y')).upper()
today, yesterday

('28OCT2022', '27OCT2022')

In [43]:
# # custom dates
# today = "27OCT2022"
# yesterday = "25OCT2022"

In [44]:
# source url for fetching the data from nseIndia.com
source_url = 'https://archives.nseindia.com/content/historical/DERIVATIVES/2022/OCT/fo{}bhav.csv.zip'.format(today)  # CHANGE-ME
source_url

'https://archives.nseindia.com/content/historical/DERIVATIVES/2022/OCT/fo28OCT2022bhav.csv.zip'

In [45]:
# fetching the zip file
# fetching the csv file 
# NOTE : the zipfile must contain only csv file nothing else
temp_data = pd.read_csv(source_url,compression='zip')

In [46]:
# Dropping the unnecessary columns
temp_data = temp_data.drop(columns=['STRIKE_PR', 'OPEN', 'OPTION_TYP', 'HIGH', 'LOW', 'TIMESTAMP'])

In [47]:
# Dropping the last column as it was empty
temp_data.drop(temp_data.columns[[9]], axis=1, inplace=True)

In [48]:
# NIFTY 100 STOCK NAMES
companies = """ADANITRANS
HAL
BEL
SBIN
BANKBARODA
ADANIPORTS
SBICARD
EICHERMOT
SIEMENS
ITC
SBILIFE
NESTLEIND
ADANIENT
CHOLAFIN
BHARTIARTL
INDUSINDBK
INDIGO
HAVELLS
TATAMOTORS
DLF
M&M
APOLLOHOSP
LT
RELIANCE
NAUKRI
CIPLA
LTI
MUTHOOTFIN
HDFCLIFE
COALINDIA
HCLTECH
HINDALCO
TATAPOWER
ATGL
ICICIGI
TATACONSUM
ICICIBANK
BIOCON
ULTRACEMCO
TATASTEEL
WIPRO
HDFCAMC
IOC
MCDOWELL-N
ASIANPAINT
BAJAJHLDNG
PIIND
BERGEPAINT
INDUSTOWER
INFY
JSWSTEEL
MARUTI
SHREECEM
TCS
ONGC
ZOMATO
BOSCHLTD
TITAN
ICICIPRULI
BANDHANBNK
COLPAL
PIDILITIND
HINDUNILVR
UPL
POWERGRID
SRF
BAJFINANCE
MARICO
MPHASIS
IRCTC
BAJAJFINSV
ADANIGREEN
AXISBANK
HEROMOTOCO
DRREDDY
DABUR
GODREJCP
VEDL
PGHH
DIVISLAB
KOTAKBANK
GLAND
GAIL
LICI
BAJAJ-AUTO
BPCL
GRASIM
HDFCBANK
TECHM
DMART
BRITANNIA
SUNPHARMA
HDFC
NTPC
TORNTPHARM
NYKAA
PAYTM
AMBUJACEM
ACC
MOTHERSON
BANKNIFTY
NIFTY
"""
companies = companies.split('\n')
len(companies)

103

In [49]:
# Inplace droppping the row with symbol as FINNIFTY
# temp_data.drop(temp_data[temp_data['SYMBOL'] == 'FINNIFTY'].index, inplace=True)

# Filtering out data based on INSTRUMENT AND SYMBOL
temp_data = temp_data.loc[temp_data['INSTRUMENT'].isin(['FUTSTK', 'FUTIDX'])]
temp_data = temp_data.loc[temp_data['SYMBOL'].isin(companies)]

In [50]:
# Filtering out data based on current month expiry date
temp_data = temp_data[temp_data['EXPIRY_DT'].astype(str).str.match(r"\d{2}-Oct-2022")]

In [51]:
# Dropping out the first column i.e. INSTRUMENT
temp_data.drop(temp_data.columns[[0]], axis=1, inplace=True)

In [52]:
# new Column names (added date with the names)
column_names = [ 'SYMBOL', 'EXPIRY_DT', 'CLOSE'+today, 'SETTLE_PR'+today, 'CONTRACTS'+today, 'VAL_INLAKH'+today,'OPEN_INT'+today, 'CHG_IN_OI'+today]

# assigning the new column names
temp_data.columns = column_names

In [53]:
temp_data

,SYMBOL,EXPIRY_DT,CLOSE28OCT2022,SETTLE_PR28OCT2022,CONTRACTS28OCT2022,VAL_INLAKH28OCT2022,OPEN_INT28OCT2022,CHG_IN_OI28OCT2022


In [54]:
# writing the today's data to temp file to refresh the index.
temp_data.to_csv('temp_data.csv', index=False)

In [55]:
# reading the today's data from the temp file.
today_data = pd.read_csv('temp_data.csv')

# reading the main data from the mainfile
main_data = pd.read_csv('fandodata.csv')


In [56]:
# removing today's data's  expiry column as not required during merge
today_data.drop(columns='EXPIRY_DT', inplace=True)

In [57]:
# merging today's data with main data
main_data = main_data.merge(today_data, how='outer', on='SYMBOL')

In [58]:
# rearrange data
header_main = ['CLOSE'+yesterday, 'SETTLE_PR'+yesterday, 'CONTRACTS'+yesterday, 'VAL_INLAKH'+yesterday,'OPEN_INT'+yesterday, 'CHG_IN_OI'+yesterday]
header_new = ['CLOSE'+today, 'SETTLE_PR'+today, 'CONTRACTS'+today, 'VAL_INLAKH'+today,'OPEN_INT'+today, 'CHG_IN_OI'+today]

for i in range(6):
    new_head = header_new[i]
    prev_head = main_data.columns.get_loc(header_main[i]) + 1

    extracted_col = main_data[new_head]
    main_data.drop(columns=new_head, inplace=True)
    main_data.insert(prev_head, new_head, extracted_col)
    

In [59]:
main_data

,SYMBOL,EXPIRY_DT,CLOSE24OCT2022,CLOSE25OCT2022,CLOSE27OCT2022,CLOSE28OCT2022,SETTLE_PR24OCT2022,SETTLE_PR25OCT2022,SETTLE_PR27OCT2022,SETTLE_PR28OCT2022,...,VAL_INLAKH27OCT2022,VAL_INLAKH28OCT2022,OPEN_INT24OCT2022,OPEN_INT25OCT2022,OPEN_INT27OCT2022,OPEN_INT28OCT2022,CHG_IN_OI24OCT2022,CHG_IN_OI25OCT2022,CHG_IN_OI27OCT2022,CHG_IN_OI28OCT2022
0,BANKNIFTY,27-Oct-2022,41270.00,41111.15,41283.30,NaN,41270.00,41111.15,41299.30,NaN,...,1283416.40,NaN,1297250,1016825,573500,NaN,-109675,-280425,-443325,NaN
1,NIFTY,27-Oct-2022,17732.55,17654.95,17721.70,NaN,17732.55,17654.95,17736.95,NaN,...,1655693.19,NaN,10138500,7402750,3711050,NaN,-423400,-2735750,-3691700,NaN
2,ACC,27-Oct-2022,2255.15,2251.05,2313.00,NaN,2255.15,2251.05,2313.00,NaN,...,34224.21,NaN,2382500,1369500,418500,NaN,-15750,-1013000,-951000,NaN
3,ADANIENT,27-Oct-2022,3307.60,3301.50,3322.05,NaN,3307.60,3301.50,3325.00,NaN,...,163000.79,NaN,6308000,2947000,335500,NaN,-75500,-3361000,-2611500,NaN
4,ADANIPORTS,27-Oct-2022,801.30,801.15,822.45,NaN,801.30,801.15,821.25,NaN,...,134318.26,NaN,21832500,9881250,1317500,NaN,-173750,-11951250,-8563750,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,TORNTPHARM,27-Oct-2022,1568.85,1582.85,1602.85,NaN,1568.85,1582.85,1599.65,NaN,...,6005.93,NaN,746000,258000,42500,NaN,-9000,-488000,-215500,NaN
87,ULTRACEMCO,27-Oct-2022,6397.00,6410.85,6434.30,NaN,6397.00,6410.85,6436.75,NaN,...,45238.86,NaN,920900,465500,66200,NaN,-33000,-455400,-399300,NaN
88,UPL,27-Oct-2022,701.35,694.85,704.25,NaN,701.35,694.85,706.45,NaN,...,56261.75,NaN,11779300,5218200,1202500,NaN,-217100,-6561100,-4015700,NaN
89,VEDL,27-Oct-2022,280.10,280.75,287.60,NaN,280.10,280.75,288.35,NaN,...,31255.29,NaN,14514200,5690050,1477150,NaN,-88350,-8824150,-4212900,NaN


In [60]:
main_data.to_csv('fandodata.csv', index=False)